In [2]:
import pandas as pd
import requests
import json
import math

In [3]:
dfs = pd.read_excel('song_data.xlsx',sheet_name=None)
artists = dfs['artists']
my_playlist = dfs['my_playlist']
artist_tracks = dfs['artist_tracks']

In [3]:
token = "BQAf3_P8u8jp5A0Cpk3X6AONBktuQd7WAeIUdKSkkeLTVOKq4egGmy-FTMKk8q7B6XyLgPRYnPDpNex1zuLrPq5iGDuq8kRePuJrcBDG_MmiHEGALAKsDN3VdXDZCg_ZUskH53Shh_yjO8H7UGIDGhGfWOEaMVPjDNCj01tq6qg7AYYlKdMCfiwavD0HTuEDZYE"

headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': 'Bearer '+token,
}

base_url = 'https://api.spotify.com/v1'

In [ ]:
related_artists = {}
for i, aid in enumerate(artists['id']):
    related_json = requests.get(f"{base_url}/artists/{aid}/related-artists",headers=headers).json()
    related_artists[aid] = [artist['id'] for artist in related_json['artists']]
    if i%10 == 0:
        print(i)

In [5]:
artists_in_library = {}
for aid in related_artists:
    ra = set(related_artists[aid]) & set(artists['id'])
    if ra:
        artists_in_library[aid] = list(ra)

In [41]:
mp_artists = my_playlist.merge(artist_tracks, left_on='id',right_on='track_id')['artist_id'].unique()
mp_related_artists = {}
for aid in mp_artists:
    ra = set(related_artists[aid]) & set(mp_artists)
    if ra:
        mp_related_artists[aid] = list(ra)

In [68]:
algo_df = pd.DataFrame(mp_related_artists.keys(),columns=['id'])
algo_df['target'] = algo_df['id'].map(mp_related_artists)
algo_df = algo_df.explode('target')
algo_df['weight'] = 1

algo_matrix = algo_df.pivot(index='target',columns='id',values='weight').fillna(0)

In [69]:
algo_matrix.to_csv('algo_adj_matrix.csv')

In [86]:
data = json.load(open('graph_test.json',))

In [87]:
nodes = pd.DataFrame(data['nodes'])
nodes['artist'] = nodes['id'].map(dict(zip(artists['id'],artists['name'])))
nodes.to_csv('nodes.csv',index=False)

In [97]:
edges = algo_df[['id','target']]
e2 = edges.copy()
edges['name'] = edges['id']
e2['name'] = edges['target']
edges = edges.append(e2)
edges.to_csv('edges.csv',index=False)

In [19]:
feats = ['id','name','danceability','speechiness','acousticness','instrumentalness','liveness','valence','energy']
tracks = dfs['tracks'][feats].copy()
tracks['key'] = 1

In [20]:
tdf = tracks.merge(tracks,on='key').drop('key',axis=1)
tdf = tdf[tdf['id_x']!=tdf['id_y']]
tdf['feat_x'] = tdf[[feat+'_x' for feat in feats][2:]].values.tolist()
tdf['feat_y'] = tdf[[feat+'_y' for feat in feats][2:]].values.tolist()
tdf['diff'] = tdf.apply(lambda x: track_diff(x['feat_x'],x['feat_y']),axis=1)
tdf = tdf[['name_x','id_x','name_y','id_y','feat_x','feat_y','diff']]
tdf = tdf[tdf['name_x']!=tdf['name_y']]
tdf['set'] = tdf[['name_x','name_y']].apply(frozenset, axis=1)
tdf = tdf.drop_duplicates(subset='set')
tdf= tdf.drop('set',axis=1)

In [21]:
def track_diff(feat_x,feat_y):
    squares = [(xy[0]-xy[1])**2 for xy in zip(feat_x,feat_y)]
    return math.sqrt(sum(squares))

In [22]:
most_diff = tdf.sort_values('diff',ascending=False)#.groupby(['name_x','name_y'],as_index=False).first()
least_diff = tdf.sort_values('diff')

In [76]:
most_diff_list = [2231504,448784,678212,581850,341844,2727380,510651,68633,131500,2202541]
least_diff_list = [611690,1033721,720578,1819454,1677363,1169504,637049,496170,2123955,1657123]
excl_list = []
least_diff[~least_diff['name_x'].isin(excl_list) & ~least_diff['name_y'].isin(excl_list)].iloc[40:60]

,name_x,id_x,name_y,id_y,feat_x,feat_y,diff
2067478,Counting Stars,1fh2TfUOtOoXEE8qQynYEC,Connect,7hi776S56F8a17xzXANdsj,"[0.663, 0.0381, 0.0649, 0.0, 0.119, 0.48, 0.712]","[0.629, 0.0573, 0.056, 0.00156, 0.104, 0.489, ...",0.043832
271679,White Leather,3ipx41kyXC11Ocp3qxOgse,High,0Ji6UviVtG3hVK86PwlQmI,"[0.76, 0.0548, 0.0132, 0.000511, 0.0682, 0.796...","[0.758, 0.0353, 0.00627, 0.00349, 0.0665, 0.75...",0.044520
396689,Perfect Places,0PGQFIxe8kC5IAdNfAOtEM,Funny Business,6hxyNgHZxJLz3xxjBlRXtN,"[0.616, 0.0561, 0.0024, 2.91e-05, 0.128, 0.521...","[0.624, 0.0486, 0.0349, 7.83e-06, 0.105, 0.507...",0.044525
361740,Destroyer,7tFa04zJGCdcRLmrXLThOe,Red Hands,3e5uxnPKYPdNjadbGXjWbP,"[0.646, 0.0334, 0.000536, 4.78e-06, 0.235, 0.6...","[0.688, 0.0268, 0.00558, 0.0, 0.223, 0.672, 0....",0.044654
2123955,17 - Single Version,4YFHvy9PsT1ExBAYsTdngn,Rad Drugz,0u2A867SnSHy7JOmQtNA3x,"[0.639, 0.0332, 0.00138, 0.0, 0.0682, 0.605, 0...","[0.612, 0.0325, 0.0197, 2.73e-06, 0.0824, 0.57...",0.044797
170604,The Way It Was,0XoGin3sawtB2HmME0Ikn3,Ride or Die (feat. Foster the People),3uJsg8xIH7OpWHxuJaMLJX,"[0.67, 0.0503, 0.0833, 3.71e-05, 0.0926, 0.579...","[0.648, 0.0335, 0.0609, 4.41e-06, 0.104, 0.556...",0.044810
1077397,Adventure of a Lifetime,4MFCXniy7I8tj0GXtf3wgC,Morocco,30lgD1UuHczwlxa7NZFeSQ,"[0.638, 0.036, 0.00205, 0.000175, 0.149, 0.53,...","[0.616, 0.0508, 0.0178, 0.000103, 0.153, 0.542...",0.044845
1169227,Let It Happen,2X485T9Z5Ly0xyaghN73ed,Rad Drugz,0u2A867SnSHy7JOmQtNA3x,"[0.602, 0.0443, 0.00462, 0.0266, 0.111, 0.577,...","[0.612, 0.0325, 0.0197, 2.73e-06, 0.0824, 0.57...",0.044956
1956343,Feel Again,6nmVeODcBpsGKx5RPv003D,Pressure To Party,65ziMiqYBpLirJJwjkhZu6,"[0.474, 0.0541, 0.0243, 3.56e-06, 0.105, 0.353...","[0.483, 0.0366, 0.000124, 4.79e-05, 0.118, 0.3...",0.045230
1147265,The Trap,0kTXtk7NSe3OzQ4FDonwyO,Take It Easy (feat. U.S. Girls & Ledinsky),4ZnfMeF25goNtKuVlpNn9Q,"[0.495, 0.0381, 0.0033, 0.00478, 0.33, 0.692, ...","[0.479, 0.0363, 0.00984, 0.0, 0.349, 0.689, 0....",0.045430


In [123]:
#adf['artists'] = adf['track_id'].map(adf.groupby('track_id').agg(lambda x: ', '.join(x)))
adf = dfs['artist_tracks'].merge(dfs['artists'],left_on='artist_id',right_on='id')[['track_id','artist_id','name']]
adf['artists']=adf['track_id'].map(adf.groupby('track_id').agg(lambda x: ', '.join(x))['name'])
adf = adf[['track_id','artist_id','artists']]

In [24]:
adf = dfs['artist_tracks'].merge(dfs['artists'],left_on='artist_id',right_on='id')[['track_id','artist_id','name']]
adf = adf.groupby('track_id').agg(lambda x: ', '.join(x))[['name']]
adf = my_playlist[['id']].merge(adf,left_on='id',right_on='track_id').rename(columns={'name':'artists'})

In [25]:
most_diff_list = [2231504,1538956,895816,678212,449387,1042860,800466,1184000,342253,511406]
least_diff_list = [818951,861562,611690,2123918,1026405,1033721,1692457,1236823,1088560,2477710]

In [78]:
tdf.loc[most_diff_list]
#in the future look at most different and same with average

,name_x,id_x,name_y,id_y,feat_x,feat_y,diff
2231504,Fell In Love With A Girl,6YK2tAEYnHgYkkC6TSziqM,Warriors,0Mcj0sP4xiyctG9kDY86j8,"[0.342, 0.116, 0.704, 7.29e-05, 0.854, 0.373, ...","[0.964, 0.1, 0.0105, 0.915, 0.0747, 0.207, 0.595]",1.577912
448784,My Eyes,34WAdNQOUeGjgFOZshew1e,Class Historian,5iaWq4djtfWZk8McqrtprO,"[0.435, 0.0309, 0.852, 2.63e-05, 0.113, 0.0831...","[0.571, 0.0515, 0.23, 0.8, 0.397, 0.966, 0.975]",1.532804
678212,Sound & Color,65wx71brAmEQz66GXXF8gI,Silvertongue,0yTJtxZJ5VLfBEHhiNUNeY,"[0.547, 0.0667, 0.848, 0.613, 0.0825, 0.175, 0...","[0.673, 0.0408, 0.00975, 0.0, 0.752, 0.841, 0....",1.454526
581850,Liability,4JyLFq2mHC8JPFTXHwYPWD,Disparate Youth,0t2QiRkpag0fAgs9zuCPlH,"[0.596, 0.149, 0.925, 0.0, 0.134, 0.334, 0.219]","[0.642, 0.0417, 0.0131, 0.825, 0.215, 0.548, 0...",1.443993
341844,D.A.N.C.E.,2hQCzcb3qyZirWzOD5Yzoj,When I Was Your Man,0nJW01T7XtvILxQgC5J7Wh,"[0.594, 0.338, 0.0463, 0.0, 0.853, 0.531, 0.961]","[0.612, 0.0434, 0.932, 0.0, 0.088, 0.387, 0.28]",1.393305
2727380,Hazy Miss Daisy,6b8nz7zFlAeNACOA1lVYG7,Clay Pigeon,1yZyPhdJF5L60qDUeuwojc,"[0.747, 0.0416, 0.888, 0.0087, 0.126, 0.773, 0...","[0.579, 0.121, 0.000339, 0.495, 0.597, 0.247, ...",1.395744
510651,The Underdog,2YRuexk6q6so8F5M69n4t1,Intro,0DAsxISzun85PbsqAfIzeC,"[0.684, 0.034, 0.0567, 0.000412, 0.703, 0.889,...","[0.617, 0.027, 0.459, 0.925, 0.128, 0.152, 0.778]",1.382543
68633,Mr. Redundant,2yPiN3T8Z0YE4uovhPyaB3,Sunflower Bean,2hXm7j5AOBwd9XWucharDY,"[0.837, 0.0482, 0.783, 0.0, 0.0901, 0.171, 0.308]","[0.419, 0.073, 0.000544, 0.591, 0.286, 0.791, ...",1.375705
131500,Get Lost,7eFOYhF4uIMOFxgGAayf36,3WW,65lTO69QCwDl3EmgU3eGKv,"[0.582, 0.0756, 0.0275, 0.0487, 0.732, 0.728, ...","[0.634, 0.0533, 0.679, 0.036, 0.0612, 0.158, 0...",1.370537
2202541,Wide Awake,7COxY38TB5HTeXRAucOEW4,my future,2ygvZOXrIeVL4xZmAWJT2C,"[0.85, 0.036, 0.113, 0.435, 0.0697, 0.897, 0.94]","[0.444, 0.062, 0.795, 0.132, 0.352, 0.0875, 0....",1.362200


In [85]:
tdf[tdf['id_y'] == '0Mcj0sP4xiyctG9kDY86j8']

,name_x,id_x,name_y,id_y,feat_x,feat_y,diff


In [94]:
a = dfs['artists']
a[a['id']=='6MKNzi38fPQCFRdWOtHqTJ']

,id,name,popularity,followers
209,6MKNzi38fPQCFRdWOtHqTJ,Erik Hassle,45,52249
